# Use Amazon SageMaker for Parameter-Efficient Fine Tuning of the ESM-2 Protein Language Model

Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
SPDX-License-Identifier: MIT-0

Note: We recommend running this notebook on a **ml.t3.medium** instance with the **Data Science 3.0** image.

### What is a Protein?

Proteins are complex molecules that are essential for life. The shape and structure of a protein determines what it can do in the body. Knowing how a protein is folded and how it works helps scientists design drugs that target it. For example, if a protein causes disease, a drug might be made to block its function. The drug needs to fit into the protein like a key in a lock. Understanding the protein's molecular structure reveals where drugs can attach. This knowledge helps drive the discovery of innovative new drugs.

![Proteins are made up of long chains of amino acids](../img/protein.png)

### What is a Protein Language Model?

Proteins are made up of linear chains of molecules called amino acids, each with its own chemical structure and properties. If we think of each amino acid in a protein like a word in a sentence, it becomes possible to analyze them using methods originally developed for analyzing human language. Scientists have trained these so-called, "Protein Language Models", or pLMs, on millions of protein sequences from thousands of organisms. With enough data, these models can begin to capture the underlying evolutionary relationships between different amino acid sequences.

It can take a lot of time and compute to train a pLM from scratch for a certain task. For example, a team at Tsinghua University [recently described](https://www.biorxiv.org/content/10.1101/2023.07.05.547496v3) training a 100 Billion-parameter pLM on 768 A100 GPUs for 164 days! Fortunately, in many cases we can save time and resources by adapting an existing pLM to our needs. This technique is called "fine-tuning" and also allows us to borrow advanced tools from other types of language modeling

### What is ESM-2?

[ESM-2](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1) is a pLM trained using unsupervied masked language modelling on 250 Million protein sequences by researchers at [Facebook AI Research (FAIR)](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1). It is available in several sizes, ranging from 8 Million to 15 Billion parameters. The smaller models are suitable for various sequence and token classification tasks. The FAIR team also adapted the 3 Billion parameter version into the ESMFold protein structure prediction algorithm. They have since used ESMFold to predict the struture of [more than 700 Million metagenomic proteins](https://esmatlas.com/about). 

ESM-2 is a powerful pLM. However, it has traditionally required multiple A100 GPU chips to fine-tune. In this notebook, we demonstrate how to use QLoRA to fine-tune ESM-2 in on an inexpensive Amazon SageMaker training instance. We will use ESM-2 to predict [subcellular localization](https://academic.oup.com/nar/article/50/W1/W228/6576357). Understanding where proteins appear in cells can help us understand their role in disease and find new drug targets. 

---
## 1. Set up environment

In [1]:
!pip install --disable-pip-version-check torch=='2.1.1+cpu' --index-url https://download.pytorch.org/whl/cpu
%pip install --upgrade sagemaker
!pip install transformers=='4.35.2' datasets=='2.15.0' s3fs=='0.4.2'

Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Using cached multiprocess-0.70.17-py311-none-any.whl.metadata (7.2 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached multiprocess-0.70.15-py311-none-any.whl.metadata (7.2 kB)
Using cached dill-0.3.7-py3-none-any.whl (115 kB)
Using cached multiprocess-0.70.15-py311-none-any.whl (135 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.18
    Uninstalling multiprocess-0.70.18:
      Successfully uninstalled multiprocess-0.70.18
ERROR: pip

In [2]:
import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()

S3_BUCKET = sagemaker_session.default_bucket()

default_bucket = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix
smu_base_path = f"s3://{default_bucket}/{default_prefix}/"
print(smu_base_path)

s3 = boto3.client("s3")
sagemaker_client = boto3.client("sagemaker")
REGION_NAME = sagemaker_session.boto_region_name

sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemak

In [3]:
import boto3
from datasets import Dataset
import json
import os
import pandas as pd
import random
import sagemaker
from sagemaker.experiments.run import Run
from sagemaker.huggingface import HuggingFace, HuggingFaceModel
from sagemaker.inputs import TrainingInput
from time import strftime
from transformers import AutoTokenizer

try:
    sagemaker_execution_role = sagemaker_session.get_execution_role()
except AttributeError:
    NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        instance_name = metadata["ResourceName"]
        domain_id = metadata.get("DomainId")
        user_profile_name = metadata.get("UserProfileName")
        space_name = metadata.get("SpaceName")
    domain_desc = sagemaker_session.sagemaker_client.describe_domain(DomainId=domain_id)
    if "DefaultSpaceSettings" in domain_desc:
        sagemaker_execution_role = domain_desc["DefaultSpaceSettings"]["ExecutionRole"]
    else:
        sagemaker_execution_role = domain_desc["DefaultUserSettings"]["ExecutionRole"]

print(f"Assumed SageMaker role is {sagemaker_execution_role}")

S3_PREFIX = "esm-loc-ft"
S3_PATH = sagemaker.s3.s3_path_join("s3://", S3_BUCKET, S3_PREFIX)
print(f"S3 path is {S3_PATH}")

EXPERIMENT_NAME = "esm-loc-ft-" + strftime("%Y-%m-%d-%H-%M-%S")
print(f"Experiment name is {EXPERIMENT_NAME}")

Assumed SageMaker role is arn:aws:iam::191384580845:role/datazone_usr_role_aromn82irdy0br_czv2v6qkj9v3jb
S3 path is s3://amazon-sagemaker-191384580845-us-east-1-3e7e9e6558a6/esm-loc-ft
Experiment name is esm-loc-ft-2025-04-29-16-32-58


Load the sagemaker package and create some service clients

---
## 2. Build Dataset

We'll use a version of the [DeepLoc-2 data set](https://services.healthtech.dtu.dk/services/DeepLoc-2.0/) to fine tune our localization model. It consists of several thousand protein sequences, each with one or more experimentally-observed location labels. This data was extracted by the DeepLoc team at Technical University of Denmark from the public [UniProt sequence database](https://www.uniprot.org/).

In [4]:
df = pd.read_csv(
    "https://services.healthtech.dtu.dk/services/DeepLoc-2.0/data/Swissprot_Train_Validation_dataset.csv"
).drop(["Unnamed: 0", "Partition"], axis=1)
df["Membrane"] = df["Membrane"].astype("int32")

# filter for sequences between 100 and 512 amino acides
df = df[df["Sequence"].apply(lambda x: len(x)).between(100, 512)]

# Remove unnecessary features
df = df[["Sequence", "Kingdom", "Membrane"]]

display(df)
print(df.groupby("Kingdom").size())
print()
print(df.groupby("Membrane").size())

,Sequence,Kingdom,Membrane
0,MAAAAAAAAAAGAAGGRGSGPGRRRHLVPGAGGEAGEGAPGGAGDY...,Metazoa,0
1,MAAAAAAAAAAGAAGGRGSGPGRRRHLVPGAGGEAGEGAPGGAGDY...,Metazoa,0
3,MAAAAAAAAATEQQGSNGPVKKSMREKAVERRNVNKEHNSNFKAGY...,Metazoa,1
4,MAAAAAAAAATNGTGGSSGMEVDAAVVPSVMACGVTGSVSVALHPL...,Metazoa,0
6,MAAAAAAAGGAALAVSTGLETATLQKLALRRKKVLGAEEMELYELA...,Metazoa,0
...,...,...,...
28295,MYYFSRVAARTFCCCIFFCLATAYSRPDRNPRKIEKKDKKFFGASK...,Fungi,0
28297,MYYHNQHQGKSILSSSRMPISSERHPFLRGNGTGDSGLILSTDAKP...,Viridiplantae,0
28298,MYYIGHPSYYRKHIEHVCFQHSGILKKRNYQKNQKKYIMKLNESAM...,Fungi,1
28300,MYYQNQHQGKNILSSSRMHITSERHPFLRGNSPGDSGLILSTDAKP...,Viridiplantae,0


Kingdom
Fungi            3614
Metazoa          8571
Other             387
Viridiplantae    3304
dtype: int64

Membrane
0    11175
1     4701
dtype: int64


This looks good, but the two membrane classes are unbalanced. We could resample our data and discard records from the majority class. However, this would reduce the total training data. Instead, we'll calculate the class weights during the training job and use them to adjust the loss.

Next, we tokenize the sequences and trim them to a max length of 512 amino acids.

In [5]:
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2, shuffle=True)
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")


def preprocess_data(examples, max_length=512):
    text = examples["Sequence"]
    encoding = tokenizer(text, truncation=True, max_length=max_length)
    encoding["labels"] = examples["Membrane"]
    return encoding


encoded_dataset = dataset.map(
    preprocess_data,
    batched=True,
    num_proc=os.cpu_count(),
    remove_columns=dataset["train"].column_names,
)

encoded_dataset.set_format("torch")
print(encoded_dataset)

Map (num_proc=2):   0%|          | 0/12700 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3176 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 12700
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3176
    })
})


Look at an example record

In [6]:
random_idx = random.randint(3, len(encoded_dataset["train"]))
example = encoded_dataset["train"][random_idx]

print(f"Viewing example record {random_idx}")
print(f"Raw sequence:\n{tokenizer.decode(example['input_ids'])}\n")
print(f"Tokenized sequence:\n{example['input_ids'].tolist()}\n")
print(f"Label:\n{example['labels']}")

Viewing example record 2350
Raw sequence:
<cls> M S G G G K G K H V G G K G G S K I G E R G Q M S H S A R A G L Q F P V G R V R R F L K A K T Q N N M R V G A K S A V Y S A A V L E Y L T A E V L E L A G N A A K D L K V K R I T P R H L Q L A I R G D E E L D T L I R A T I A G G G V L P H I N K Q L L I R T K E K Y P E E E E I I <eos>

Tokenized sequence:
[0, 20, 8, 6, 6, 6, 15, 6, 15, 21, 7, 6, 6, 15, 6, 6, 8, 15, 12, 6, 9, 10, 6, 16, 20, 8, 21, 8, 5, 10, 5, 6, 4, 16, 18, 14, 7, 6, 10, 7, 10, 10, 18, 4, 15, 5, 15, 11, 16, 17, 17, 20, 10, 7, 6, 5, 15, 8, 5, 7, 19, 8, 5, 5, 7, 4, 9, 19, 4, 11, 5, 9, 7, 4, 9, 4, 5, 6, 17, 5, 5, 15, 13, 4, 15, 7, 15, 10, 12, 11, 14, 10, 21, 4, 16, 4, 5, 12, 10, 6, 13, 9, 9, 4, 13, 11, 4, 12, 10, 5, 11, 12, 5, 6, 6, 6, 7, 4, 14, 21, 12, 17, 15, 16, 4, 4, 12, 10, 11, 15, 9, 15, 19, 14, 9, 9, 9, 9, 12, 12, 2]

Label:
0


Finally, we upload the processed training, test, and validation data to S3.

In [ ]:
train_s3_uri = smu_base_path + "esm/data/train"
test_s3_uri = smu_base_path + "esm/data/test"

encoded_dataset["train"].save_to_disk(train_s3_uri)
encoded_dataset["test"].save_to_disk(test_s3_uri)

Saving the dataset (0/1 shards):   0%|          | 0/12700 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3176 [00:00<?, ? examples/s]

---
## 3. Train model in SageMaker

Let's try a few different approaches to improving the efficiency of our fine-tuning job.

### Gradient Accumulation

Gradient accumulation is a training technique that allows models to simulate training on larger batch sizes. Typically, the batch size - the number of samples used to calculate the gradient in one training step - is limited by the GPU memory capacity. With gradient accumulation, the model calculates gradients on smaller batches first. Then, instead of updating the model weights right away, the gradients get accumulated over multiple small batches. Once the accumulated gradients equal the target larger batch size, the optimization step is performed to update the model. This lets models train with effectively bigger batches without exceeding the GPU memory limit. However, extra computation is needed for the smaller batch forward and backward passes. So increased batch sizes via gradient accumulation can slow down training, especially if too many accumulation steps are used. The aim is to maximize GPU usage but avoid excessive slowdowns from too many extra gradient computation steps.

### Gradient Checkpointing

Gradient checkpointing is a technique that reduces the memory needed during training while keeping the computational time reasonable. Large neural networks take up a lot of memory because they have to store all the intermediate values from the forward pass in order to calculate the gradients during the backward pass. This can cause memory issues. One solution is to not store these intermediate values, but then they have to be recalculated during the backward pass, which takes a lot of time. 

Gradient checkpointing provides a balanced approach. It saves only some of the intermediate values, called "checkpoints," and recalculates the others as needed. So it uses less memory than storing everything, but also less computation than recalculating everything. By strategically selecting which activations to checkpoint, gradient checkpointing enables large neural networks to be trained with manageable memory usage and computation time. This important technique makes it feasible to train very large models that would otherwise run into memory limitations.

### Low-Rank Adaptation of Large Language Models (LoRA)

Large language models like ESM-2 can contain billions of parameters that are expensive to train and run. [Researchers](https://www.biorxiv.org/content/10.1101/2023.07.05.547496v3) have developed a new training method called Low-Rank Adaptation (LoRA) to make fine-tuning these huge models more efficient. 

The key idea behind LoRA is that when fine-tuning a model for a specific task, you don't need to update all of the original parameters. Instead, LoRA adds new smaller matrices to the model that transform the inputs and outputs. Only these smaller matrices are updated during fine-tuning, which is much faster and uses less memory. The original model parameters stay frozen. 

After fine-tuning with LoRA, the small adapted matrices can be merged back into the original model. Or they can be kept separate if you want to quickly fine-tune the model for other tasks without forgetting previous ones. Overall, LoRA allows large language models to be efficiently adapted to new tasks at a fraction of the usual cost.

In [8]:
hyperparameters = {
    "model_id": "facebook/esm2_t33_650M_UR50D",
    "epochs": 1,
    "per_device_train_batch_size": 8,
    "gradient_accumulation_steps": 4,
    "use_gradient_checkpointing": True,
    "lora": True,
}

metric_definitions = [
    {"Name": "epoch", "Regex": "'epoch': ([0-9.]*)"},
    {
        "Name": "max_gpu_mem",
        "Regex": "Max GPU memory use during training: ([0-9.e-]*) MB",
    },
    {"Name": "train_loss", "Regex": "'loss': ([0-9.e-]*)"},
    {
        "Name": "train_samples_per_second",
        "Regex": "'train_samples_per_second': ([0-9.e-]*)",
    },
    {"Name": "eval_loss", "Regex": "'eval_loss': ([0-9.e-]*)"},
    {"Name": "eval_accuracy", "Regex": "'eval_accuracy': ([0-9.e-]*)"},
]

hf_estimator = HuggingFace(
    base_job_name="esm-2-membrane-ft",
    entry_point="lora-train.py",
    source_dir="scripts",
    instance_type="ml.p3.2xlarge",
    instance_count=1,
    transformers_version="4.28",
    pytorch_version="2.0",
    py_version="py310",
    output_path=f"{S3_PATH}/output",
    role=sagemaker_execution_role,
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    checkpoint_local_path="/opt/ml/checkpoints",
    sagemaker_session=sagemaker_session,
    tags=[{"Key": "project", "Value": "esm-fine-tuning"}],
)

sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.SecurityGroupIds


In [10]:
with Run(
    experiment_name=EXPERIMENT_NAME,
    sagemaker_session=sagemaker_session,
) as run:
    hf_estimator.fit(
        {
            "train": TrainingInput(s3_data=train_s3_uri, input_mode="File"),
            "test": TrainingInput(s3_data=test_s3_uri, input_mode="File"),
        },
        wait=False,
    )

You can view metrics and debugging information for this run in SageMaker Experiments.

In [11]:
from sagemaker.analytics import ExperimentAnalytics

training_job_details = hf_estimator.latest_training_job.describe()
print(f"Training job name: {training_job_details.get('TrainingJobName')}")
print(f"Training job status: {training_job_details.get('TrainingJobStatus')}")
print(f"Training job output: {training_job_details.get('ModelArtifacts')}")

search_expression = {
    "Filters": [
        {
            "Name": "DisplayName",
            "Operator": "Contains",
            "Value": "Training",
        }
    ],
}

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sagemaker_session,
    experiment_name=EXPERIMENT_NAME,
    search_expression=search_expression,
)

trial_component_analytics.dataframe().T

Training job name: esm-2-membrane-ft-2025-04-29-16-39-14-502
Training job status: InProgress
Training job output: None


,0
TrialComponentName,esm-2-membrane-ft-2025-04-29-16-39-14-502-aws-...
DisplayName,esm-2-membrane-ft-2025-04-29-16-39-14-502-aws-...
SourceArn,arn:aws:sagemaker:us-east-1:191384580845:train...
SageMaker.ImageUri,763104351884.dkr.ecr.us-east-1.amazonaws.com/h...
SageMaker.InstanceCount,1.0
SageMaker.InstanceType,ml.p3.2xlarge
SageMaker.VolumeSizeInGB,30.0
epochs,1.0
gradient_accumulation_steps,4.0
lora,true


The following table compares the different training methods we discussed above and their effect on the run time, accuracy, and GPU memory requirements of our job.

| Configuration | Billable time (sec) | Evaluation Accuracy | Max GPU Memory Usage (GB)
| ----------- | ----------- | ----------- | ----------- |
| Base 650M model | 1698 | 0.91 | 22.6 |
| Base + Gradient Accumulation | 1266 | 0.90 | 17.8 |
| Base + Gradient Checkpointing | 1753 | 0.91 | 10.2|
| Base + LoRA | 1362 | 0.90 | 18.6 |


All of the methods produced models with high evaluation accuracy. Using LoRA decreased the run time (and cost) by 32% and using gradient checkpointing decreased the maximum GPU memory usage by 60%. Depending on our constraints (cost, time, hardware) one of these approaches may make more sense than another.

Each of these methods perform well by themselves, but what happens when we use them in combination? Here are the results:

| Configuration | Billable time (sec) | Evaluation Accuracy | Max GPU Memory Usage (GB)
| ----------- | ----------- | ----------- | ----------- |
| Base + LoRA + GA + GC | 689 | 0.80 | 3.3 |

In this case, we see a 12% reduction in accuracy. However, the run time improvements are very close to what we saw with LoRA by itself. More importantly, we've reduced the GPU memory use by 85%! This is a massive decrease that allows us to train on a wide range of cost-effective instance types.


---
## 4. Deploy Model as Real-Time Inference Endpoint

Finally, let's deploy our trained model to an inference endpoint and test it against some protein sequences with known subcellular localization. In this case, we'll load a previously-trained version of the model saved on the public HuggingFace model hub.

In [ ]:
%%time

hub = {"HF_MODEL_ID": "bloyal/esm2_650M_membrane_loc", "HF_TASK": "text-classification"}

hf_model = HuggingFaceModel(
    env=hub,
    role=sagemaker_execution_role,
    transformers_version="4.28",
    pytorch_version="2.0",
    py_version="py310",
)

predictor = hf_model.deploy(
    initial_instance_count=1,
    instance_type="ml.r5.2xlarge",
    role=sagemaker_execution_role,
)

sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
--

Try running some known proteins

In [ ]:
# Example cell membrane proteins
glp_1_receptor = "MAGAPGPLRLALLLLGMVGRAGPRPQGATVSLWETVQKWREYRRQCQRSLTEDPPPATDLFCNRTFDEYACWPDGEPGSFVNVSCPWYLPWASSVPQGHVYRFCTAEGLWLQKDNSSLPWRDLSECEESKRGERSSPEEQLLFLYIIYTVGYALSFSALVIASAILLGFRHLHCTRNYIHLNLFASFILRALSVFIKDAALKWMYSTAAQQHQWDGLLSYQDSLSCRLVFLLMQYCVAANYYWLLVEGVYLYTLLAFSVLSEQWIFRLYVSIGWGVPLLFVVPWGIVKYLYEDEGCWTRNSNMNYWLIIRLPILFAIGVNFLIFVRVICIVVSKLKANLMCKTDIKCRLAKSTLTLIPLLGTHEVIFAFVMDEHARGTLRFIKLFTELSFTSFQGLMVAILYCFVNNEVQLEFRKSWERWRLEHLHIQRDSSMKPLKCPTSSLSSGATAGSSMYTATCQASCS"
pd1 = "MQIPQAPWPVVWAVLQLGWRPGWFLDSPDRPWNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQTDKLAAFPEDRSQPGQDCRFRVTQLPNGRDFHMSVVRARRNDSGTYLCGAISLAPKAQIKESLRAELRVTERRAEVPTAHPSPSPRPAGQFQTLVVGVVGGLLGSLVLLVWVLAVICSRAARGTIGARRTGQPLKEDPSAVPVFSVDYGELDFQWREKTPEPPVPCVPEQTEYATIVFPSGMGTSSPARRGSADGPRSAQPLRPEDGHCSWPL"
rit1 = "MDSGTRPVGSCCSSPAGLSREYKLVMLGAGGVGKSAMTMQFISHRFPEDHDPTIEDAYKIRIRIDDEPANLDILDTAGQAEFTAMRDQYMRAGEGFIICYSITDRRSFHEVREFKQLIYRVRRTDDTPVVLVGNKSDLKQLRQVTKEEGLALAREFSCPFFETSAAYRYYIDDVFHALVREIRRKEKEAVLAMEKKSKPKNSVWKRLKSPFRKKKDSVT"

# Example non-cell membrane proteins
tubulin_beta_1 = "MREIVHIQIGQCGNQIGAKFWEMIGEEHGIDLAGSDRGASALQLERISVYYNEAYGRKYVPRAVLVDLEPGTMDSIRSSKLGALFQPDSFVHGNSGAGNNWAKGHYTEGAELIENVLEVVRHESESCDCLQGFQIVHSLGGGTGSGMGTLLMNKIREEYPDRIMNSFSVMPSPKVSDTVVEPYNAVLSIHQLIENADACFCIDNEALYDICFRTLKLTTPTYGDLNHLVSLTMSGITTSLRFPGQLNADLRKLAVNMVPFPRLHFFMPGFAPLTAQGSQQYRALSVAELTQQMFDARNTMAACDLRRGRYLTVACIFRGKMSTKEVDQQLLSVQTRNSSCFVEWIPNNVKVAVCDIPPRGLSMAATFIGNNTAIQEIFNRVSEHFSAMFKRKAFVHWYTSEGMDINEFGEAENNIHDLVSEYQQFQDAKAVLEEDEEVTEEAEMEPEDKGH"
p53 = "MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD"
adh5 = "MANEVIKCKAAVAWEAGKPLSIEEIEVAPPKAHEVRIKIIATAVCHTDAYTLSGADPEGCFPVILGHEGAGIVESVGEGVTKLKAGDTVIPLYIPQCGECKFCLNPKTNLCQKIRVTQGKGLMPDGTSRFTCKGKTILHYMGTSTFSEYTVVADISVAKIDPLAPLDKVCLLGCGISTGYGAAVNTAKLEPGSVCAVFGLGGVGLAVIMGCKVAGASRIIGVDINKDKFARAKEFGATECINPQDFSKPIQEVLIEMTDGGVDYSFECIGNVKVMRAALEACHKGWGVSVVVGVAASGEEIATRPFQLVTGRTWKGTAFGGWKSVESVPKLVSEYMSKKIKVDEFVTHNLSFDEINKAFELMHSGKSIRTVVKI"

sample = {"inputs": glp_1_receptor}
predictor.predict(sample)

---
## 5. Clean  up

Delete endpoint

In [ ]:
try:
    predictor.delete_endpoint()
except:
    pass

Delete S3 data

In [ ]:
boto_session = boto3.session.Session()
bucket = boto_session.resource("s3").Bucket(S3_BUCKET)
bucket.objects.filter(Prefix=S3_PREFIX).delete()